# CS348 – Café Workflow Optimization Project
## Students:
- Faris Ali Alduraibi (421107654)
- Saleh Saed Alghool (422117042)
- Suliman Saleh Alduraibi (431108067)
- Hamad Alsuhaibani (431109462)

This notebook demonstrates two optimization methods for minimizing café task makespan:
- Mixed Integer Programming (MIP)
- Genetic Algorithm (GA)


In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from pulp import *

np.random.seed(42)
random.seed(42)
orders = [f"O{i+1}" for i in range(10)]
tasks = ['Espresso', 'Blending', 'Heating']
resources = ['Espresso_1', 'Espresso_2', 'Blender', 'Oven', 'Barista']
durations = np.random.randint(2, 6, size=(10, 3))
df = pd.DataFrame({
    'Order': np.repeat(orders, 3),
    'Task': tasks * 10,
    'Duration': durations.flatten()
})
resource_map = {
    'Espresso': ['Espresso_1', 'Espresso_2'],
    'Blending': ['Blender'],
    'Heating': ['Oven']
}

## Mixed Integer Programming (MIP)

In [2]:
# MIP model
model = LpProblem("Cafe_Scheduling", LpMinimize)
assign_vars = LpVariable.dicts("assign", [(i, r) for i in df.index for r in resource_map[df.loc[i, 'Task']]], cat='Binary')
start_vars = LpVariable.dicts("start", df.index, lowBound=0)
makespan = LpVariable("makespan", lowBound=0)
model += makespan
for i in df.index:
    model += lpSum(assign_vars[i, r] for r in resource_map[df.loc[i, 'Task']]) == 1
    model += start_vars[i] + df.loc[i, 'Duration'] <= makespan
M = 1000
for r in resources:
    task_idx = [i for i in df.index if r in resource_map[df.loc[i, 'Task']]]
    for i in task_idx:
        for j in task_idx:
            if i < j:
                model += start_vars[i] + df.loc[i, 'Duration'] <= start_vars[j] + M*(2 - assign_vars[i, r] - assign_vars[j, r])
                model += start_vars[j] + df.loc[j, 'Duration'] <= start_vars[i] + M*(2 - assign_vars[i, r] - assign_vars[j, r])
model.solve()
mip_schedule = pd.DataFrame([
    (df.loc[i, 'Order'], df.loc[i, 'Task'], r, start_vars[i].varValue, df.loc[i, 'Duration'])
    for i in df.index for r in resource_map[df.loc[i, 'Task']] if assign_vars[i, r].varValue == 1
], columns=['Order', 'Task', 'Resource', 'StartTime', 'Duration']).sort_values(by='StartTime')
mip_makespan = value(makespan)
mip_schedule

,Order,Task,Resource,StartTime,Duration
0,O1,Espresso,Espresso_2,0.0,4
27,O10,Espresso,Espresso_2,0.0,3
26,O9,Heating,Oven,0.0,3
25,O9,Blending,Blender,0.0,3
24,O9,Espresso,Espresso_2,0.0,5
23,O8,Heating,Oven,0.0,5
22,O8,Blending,Blender,0.0,3
21,O8,Espresso,Espresso_2,0.0,2
19,O7,Blending,Blender,0.0,4
18,O7,Espresso,Espresso_2,0.0,5


## Genetic Algorithm (GA)

In [3]:
POP_SIZE = 30
N_GENERATIONS = 50
MUTATION_RATE = 0.1
task_options = [resource_map[df.loc[i, 'Task']] for i in df.index]
def create_individual(): return [random.choice(task_options[i]) for i in range(len(df.index))]
def evaluate(ind):
    timeline = {r: 0 for r in resources}
    starts = []
    for i, r in enumerate(ind):
        start = timeline[r]
        timeline[r] += df.loc[i, 'Duration']
        starts.append(start)
    return max(timeline.values()), starts
def mutate(ind): return [random.choice(task_options[i]) if random.random() < MUTATION_RATE else r for i, r in enumerate(ind)]
def crossover(p1, p2): return p1[:len(p1)//2] + p2[len(p1)//2:]
population = [create_individual() for _ in range(POP_SIZE)]
best_makespan, best_solution, best_start_times = float('inf'), None, None
for _ in range(N_GENERATIONS):
    evaluated = [(evaluate(ind), ind) for ind in population]
    evaluated.sort(key=lambda x: x[0][0])
    population = [ind for (_, ind) in evaluated[:10]]
    if evaluated[0][0][0] < best_makespan:
        best_makespan = evaluated[0][0][0]
        best_solution = evaluated[0][1]
        best_start_times = evaluated[0][0][1]
    while len(population) < POP_SIZE:
        c = mutate(crossover(*random.sample(population, 2)))
        population.append(c)
ga_schedule = pd.DataFrame({
    'Order': df['Order'],
    'Task': df['Task'],
    'Resource': best_solution,
    'StartTime': best_start_times,
    'Duration': df['Duration']
}).sort_values(by='StartTime')
ga_makespan = best_makespan
ga_schedule

,Order,Task,Resource,StartTime,Duration
0,O1,Espresso,Espresso_1,0,4
1,O1,Blending,Blender,0,5
2,O1,Heating,Oven,0,2
18,O7,Espresso,Espresso_2,0,5
5,O2,Heating,Oven,2,5
3,O2,Espresso,Espresso_1,4,4
4,O2,Blending,Blender,5,4
24,O9,Espresso,Espresso_2,5,5
8,O3,Heating,Oven,7,4
6,O3,Espresso,Espresso_1,8,2


## Comparison of MIP vs GA

In [4]:
comparison = pd.DataFrame({
    'Method': ['MIP', 'Genetic Algorithm'],
    'Makespan': [mip_makespan, ga_makespan],
    'Approach': ['Exact', 'Heuristic'],
    'Scalability': ['Low', 'High']
})
comparison

,Method,Makespan,Approach,Scalability
0,MIP,13.0,Exact,Low
1,Genetic Algorithm,41.0,Heuristic,High


## Enhanced Genetic Algorithm (GA) with Convergence Visualization

In [5]:
# --- Enhanced GA with convergence chart ---POP_SIZE = 30N_GENERATIONS = 50MUTATION_RATE = 0.1task_options = [resource_map[df.loc[i, 'Task']] for i in df.index]def create_individual(): return [random.choice(task_options[i]) for i in range(len(df.index))]def evaluate(ind):     timeline = {r: 0 for r in resources}    starts = []    for i, r in enumerate(ind):        start = timeline[r]        timeline[r] += df.loc[i, 'Duration']        starts.append(start)    return max(timeline.values()), startsdef mutate(ind): return [random.choice(task_options[i]) if random.random() < MUTATION_RATE else r for i, r in enumerate(ind)]def crossover(p1, p2): return p1[:len(p1)//2] + p2[len(p1)//2:]population = [create_individual() for _ in range(POP_SIZE)]best_makespan, best_solution, best_start_times = float('inf'), None, Noneconvergence = []for _ in range(N_GENERATIONS):    evaluated = [(evaluate(ind), ind) for ind in population]    evaluated.sort(key=lambda x: x[0][0])    population = [ind for (_, ind) in evaluated[:10]]    best_gen = evaluated[0][0][0]    convergence.append(best_gen)    if best_gen < best_makespan:        best_makespan = best_gen        best_solution = evaluated[0][1]        best_start_times = evaluated[0][0][1]    while len(population) < POP_SIZE:        c = mutate(crossover(*random.sample(population, 2)))        population.append(c)ga_schedule = pd.DataFrame({    'Order': df['Order'],    'Task': df['Task'],    'Resource': best_solution,    'StartTime': best_start_times,    'Duration': df['Duration']}).sort_values(by='StartTime')# Convergence plotplt.figure(figsize=(10,4))plt.plot(convergence, marker='o')plt.title('GA Convergence Over Generations')plt.xlabel('Generation')plt.ylabel('Best Makespan')plt.grid(True)plt.tight_layout()plt.show()ga_makespan = best_makespanga_schedule

## Convergence and Performance Visualization

To evaluate the performance of the Genetic Algorithm, we tracked the best makespan found in each generation. This convergence curve helps visualize how quickly the algorithm stabilizes and whether it continues to find better solutions over time. A flat line indicates early convergence, while a declining curve would imply ongoing improvement.

In this case, the convergence chart shows the GA quickly reached a stable makespan, indicating limited diversity or a local optimum.

Such visualization supports analysis of optimization behavior, efficiency, and areas for improvement (e.g., mutation tuning, population size).